In [ ]:
"""
GIT É UMA MEEERRRRDAAAA
"""

In [1]:
import os
import json
import tweepy
# import pandas as pd

In [2]:
# Leitura das chaves
with open("keys.txt","r") as text:
    text = text.read()
    keys = json.loads(text)

print(keys.keys())

dict_keys(['API_KEY', 'API_KEY_SECRET', 'ACCESS_TOKEN', 'ACCESS_TOKEN_SECRET', 'BEARER_TOKEN'])


In [3]:
def conect_api():
    auth = tweepy.OAuth1UserHandler(
      keys['API_KEY'], 
      keys['API_KEY_SECRET'], 
      keys['ACCESS_TOKEN'], 
      keys['ACCESS_TOKEN_SECRET'])
    
    return tweepy.API(auth)

In [4]:
api = conect_api()

### Verificação das credenciais

In [5]:
"""
my_credentials = api.verify_credentials()
my_credentials
"""

### testando a API com uma busca simples de tweets publicados com a hashtag pantanal

In [6]:
"""
query = "pantanal"
locale = ""
language = "pt"
"""

### Usando o método search

In [7]:
"""
tweets = api.search_tweets(q = query, lang = language, tweet_mode = "extended")

for tweet in tweets:
    try:
        print(tweet.retweeted_status.full_text)
        print("=====")
    except AttributeError:
        print(tweet.full_text)
        print("=====")
"""

## capturando tweets com o uso de cursores

#### LISTAGEM DE RESUATRANTES

In [5]:
restaurantes = {"mcdonalds","méqui","burger king", "girafas", "subway",
                "madero", "starbucks", "pizza hut","pizzahut", "dominos",
                "fogo de chao", "habibs"}

#### SENTIMENTOS

In [6]:
sentimentos_positivos = {"😍", "🥰", "😋","🤩","😀","😇"}

In [7]:
sentimentos_negativos = {"🤮","🤢","🤬","😡","😠","💩","😣"}

### MINERACAO DE DADOS

In [8]:
publicacoes = []

In [9]:
for restaurante in restaurantes:
    query = restaurante

    try:
        for status in tweepy.Cursor(api.search_tweets, q = query, lang="pt",
                                    count=200).items(10000):
            publicacoes.append(status)
    except:
        print("limite de requisicao estourado!")

limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!


In [10]:
len(publicacoes)

16669

In [62]:
"""
for pub in publicacoes:
    print(pub.text)
    print(pub.created_at)
    print("-------")
""";

### passando os principais dados para um DataFrame

In [11]:
def cria_data_frame():
    from pandas import DataFrame
    
    id_autor, texto, data_publicacao = [], [], []

    for pub in publicacoes:
        id_autor.append(pub.id_str)
        texto.append(pub.text)
        data_publicacao.append(pub.created_at)
    
    main_data = DataFrame({"id_autor":id_autor, "texto":texto ,
                           "data_publicacao":data_publicacao })
    
    return main_data


In [12]:
df = cria_data_frame()

In [13]:
df.head()

,id_autor,texto,data_publicacao
0,1534617246508752896,Hj comi o bibsfirra burguer do habibs:\nAprese...,2022-06-08 19:23:58+00:00
1,1534616823576023045,Orando pra Deus reestabelecer a gargantinha do...,2022-06-08 19:22:17+00:00
2,1534615082650456067,@gabrielaanaaa Kkkkk ah para rainha do Habibs,2022-06-08 19:15:22+00:00
3,1534611991133921281,Eu tô muito desacreditado que paguei 15.90 num...,2022-06-08 19:03:05+00:00
4,1534610220114141185,Eu tô com muita vontade de ir num rodízio de e...,2022-06-08 18:56:02+00:00


### salvando os dados

In [17]:
# separando as colunas pot \t para não dar problemar ao carregar os dados em um novo df
df.to_csv("raw-data/coleta-2.csv", sep="\t", index=False)

In [8]:
# test de carregamento
# df = pd.read_csv("raw-data/coleta-1.csv", delimiter="\t")

In [16]:
df.data_publicacao.min(), df.data_publicacao.max()

(Timestamp('2022-05-31 01:43:41+0000', tz='UTC'),
 Timestamp('2022-06-08 19:27:52+0000', tz='UTC'))